In [18]:
import os
import chromedriver_autoinstaller # chromedriver auto update
import importlib # module reload
import time
import schedule
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from webdriver_manager.chrome import ChromeDriverManager
from global_function import create_logger
from global_function import select_value_from_dropdown
from global_function import set_driver, page_back, get_weekday, check_alert
from reload_module import reload_module

In [19]:
try:
    os.chdir(r'c:\\Users\\monan\\++python\\selenium')
    chromedriver_autoinstaller.install() # auto update
    reload_module('global_function')
    logger = create_logger('benepia') # create logs
except Exception as e:
    raise Exception(e)


In [1]:
class auto_reservation:
    def __init__(self, id, pw, instanceNm):
        self.id = id
        self.pw = pw
        self.instanceNm = instanceNm
        # logger = create_logger(self.instanceNm)

    # 희망날짜 입력
    def get_dates(self, desired_dates):
        desired_dates = self.desired_dates 
        years = [desired_date[:4] for desired_date in desired_dates]
        months = [desired_date[4:6] if desired_date[4] != 0 else desired_date[5] for desired_date in desired_dates]
        dates = [desired_date[6:8] if desired_date[6] != 0 else desired_date[7] for desired_date in desired_dates]
        return zip(years, months, dates)

    # url 접속 
    def get_url(self, driver, url):
        id = self.id
        pw = self.pw
        driver.get(url)
        # if login 입력창 뜨면; 아래 진행
        if driver.find_element(By.XPATH, '//*[@id="loginId"]').is_displayed():
            try:
                logger.info('로그인합니다.')
                driver.find_element(By.XPATH, '//*[@id="loginId"]').click()
                (
                ActionChains(driver).send_keys(id).send_keys(Keys.TAB)
                .send_keys(pw).send_keys(Keys.TAB).send_keys(Keys.RETURN).perform()
                )
                driver, alert = check_alert(logger, driver)
                if alert is None:
                    logger.info('로그인 성공')
                else:
                    alert.accept()
        
            except Exception as e:
                logger.info(f'{Exception(e)}')
                raise Exception(e)
            
            finally:
                return driver
        
        else:
            return driver

    # room type 확인
    def check_hotel(self, driver, condoNm, roomNm):
        current_page = 1

        ele = driver.find_element(By.ID, 'condoNm')
        driver.execute_script("arguments[0].setAttribute('value',arguments[1])", ele, condoNm) 
        driver.find_element(By.XPATH, '//*[@id="contents"]/section/div[1]/table/tbody/tr[2]/td[3]/a').click()

        last_page = int(driver.find_element(By.CLASS_NAME, 'paging').text.split('마지막')[0][-1]) # '123마지막' 
        logger.info(f'[{condoNm}-{roomNm}] 정보를 조회합니다.')

        try:
            while (current_page <= last_page): # 전체 page에 대해 반복 탐색
                # ele = driver.find_element(By.ID, 'condoNm')

                # if ele.get_attribute('value') != '': # 입력 칸이 이미 다른 값으로 채워진 경우, 값 대체
                # driver.execute_script("arguments[0].setAttribute('value',arguments[1])", ele, condoNm) 
                driver.find_element(By.XPATH, '//*[@id="contents"]/section/div[1]/table/tbody/tr[2]/td[3]/a').click()

                if roomNm == '전체': # roomNm 입력안했으면 전체 결과 모두 조회
                    ele_hotel_names = driver.find_elements(By.XPATH, '//*[@id="contents"]/section/div[2]/table/tbody/tr/td[4]')
                    
                    for ele in ele_hotel_names:
                        logger.info(f'[{condoNm}-{roomNm}] 평형: {ele.find_element(By.XPATH, "./../td[5]").text}, 비수기 주중: {ele.find_element(By.XPATH, "./../td[7]").text}, 비수기 주말: {ele.find_element(By.XPATH, "./../td[8]").text}, 성수기 주중: {ele.find_element(By.XPATH, "./../td[9]").text}, 성수기 주말: {ele.find_element(By.XPATH, "./../td[10]").text}')
                    
                    if current_page == last_page:
                        
                        return driver
                            
                    else:
                        current_page += 1
                        driver.execute_script(f'go_page("{current_page}", "null")')
                    # current_page += 1
                    # driver.execute_script(f'go_page("{current_page}", "null")')
                    # time.sleep(1)

                elif roomNm != '전체': # roomNm 입력했으면 해당 결과만 조회 + 클릭
                    ele_hotel_names = driver.find_elements(By.XPATH, '//*[@id="contents"]/section/div[2]/table/tbody/tr/td[4]')
                    
                    for ele in ele_hotel_names:
                        if ele.text == roomNm:
                            logger.info(f'[{condoNm}-{roomNm}] 평형: {ele.find_element(By.XPATH, "./../td[5]").text}, 비수기 주중: {ele.find_element(By.XPATH, "./../td[7]").text}, 비수기 주말: {ele.find_element(By.XPATH, "./../td[8]").text}, 성수기 주중: {ele.find_element(By.XPATH, "./../td[9]").text}, 성수기 주말: {ele.find_element(By.XPATH, "./../td[10]").text}')
                            ele.find_element(By.XPATH, './../td[6]').click() # click까지
                            break
                        
                        else:
                            if current_page == last_page:
                                
                                return driver
                            
                            else:
                                current_page += 1
                                driver.execute_script(f'go_page("{current_page}", "null")')
                
        except Exception as e:
            logger.error(Exception(e))
        
        finally:

            return driver
        


    # year, month, roomNm 선택
    def select(self, driver, year, month, condoNm, roomNm): 
        if isinstance(year, int) | isinstance(month, int) | isinstance(roomNm, int):
            raise TypeError('Input(year, month, roomName) must be string type')
        else:
            try:
                logger.info(f'[{condoNm}-{roomNm}] {month}월 내 객실의 사용 가능한 날짜를 조회합니다.')
                el_year = driver.find_element(By.XPATH, '//*[@id="searchYear"]')
                el_month = driver.find_element(By.XPATH, '//*[@id="searchMonth"]')
                el_room = driver.find_element(By.XPATH, '//*[@id="roomNo"]')
                select_value_from_dropdown(logger, el_year, year)
                select_value_from_dropdown(logger, el_month, month)
                select_value_from_dropdown(logger, el_room, roomNm)
                driver.find_element(By.CLASS_NAME, 'btn-type2').click()
                driver, alert = check_alert(logger, driver) # check alert
                if alert is not None:
                    alert.accept()
                # # 잘못된 날짜(월) 선택으로 인해 초기화될 때
                # el_year = driver.find_element(By.XPATH, '//*[@id="searchYear"]')
                # el_month = driver.find_element(By.XPATH, '//*[@id="searchMonth"]')
                # year = Select(el_year).first_selected_option.text # 선택된 option 값
                # month = Select(el_month).first_selected_option.text # 선택된 option 값
                
            except Exception as e:
                logger.error(Exception(e))
                raise Exception(e)

            finally:
                return driver

    # 날짜 확인 및 예약
    def get_available_dates(self, driver, year, month, condoNm, roomNm, desired_dates, available_dates = []):
        try:
            month = month.zfill(2)
            desired_dates = [desired_dates] if isinstance(desired_dates, str) else desired_dates
            ele_check_avail = driver.find_elements(By.XPATH, '//*[@id="contents"]/article/section[2]/div/div[2]/div[2]/ul[2]/li/button')
            
            # 예약가능날짜 출력
            for ele in ele_check_avail:
                if ele.text == '신청':
                    date = ele.find_element(By.XPATH, './../span').text 
                    available_dates.append(date.zfill(2))                        
            
            logger.info('예약가능날짜: %s' %([f'{year}{month}{date.zfill(2)}({get_weekday(int(year),int(month),int(date))})' for date in available_dates]))
            
        except Exception as e:
            logger.error(Exception(e))
            raise Exception(e)

        finally:
            return driver, available_dates

    def make_reservation(self,driver,condoNm,roomNm,available_dates,desired_dates,deptNm='품질Data',areaCd='이천',telNum='010-8487-8708',roomAmt=1, adultNum=2):
        available_dates = [available_dates] if isinstance(available_dates, str) else available_dates
        # 입력 양식 기입
        def fillout(driver):
            try:
                driver.switch_to.window(driver.window_handles[1]) # switch to pop-up
                driver.find_element(By.ID, 'deptNm').send_keys(deptNm)
                select_value_from_dropdown(logger, driver.find_element(By.ID, 'areaCd'), str(areaCd))
                driver.find_element(By.ID, 'telNum').send_keys(telNum)
                driver.find_element(By.ID, 'roomAmt').send_keys(roomAmt)
                select_value_from_dropdown(logger, driver.find_element(By.ID, 'useAdultCnt'), str(adultNum))
                driver.find_element(By.ID, 'terms').click()
                # 신청하기 버튼 클릭
                driver.find_element(By.XPATH, '//*[@id="popC"]/div[4]/a').click()
            
            except Exception as e:
                logger.error(Exception(e))
                raise Exception(e)
            
            finally:
                return driver
        
        # 희망날짜와 같으면 예약(우선순위 고려)
        try:
            flag = False
            for index, date in list(enumerate(desired_dates)):
                month = date[4:6] 
                date = date[-2:] # 뒤에 날짜만
                logger.info(f'date : {date}')
                logger.info(f'available_dates : {available_dates}')
                
                ## if dates in available dates, proceed
                if date in available_dates: 
                    logger.info(f'날짜: [{condoNm}-{roomNm}] {int(month)}월{int(date)}일 예약가능합니다.')
                    flag = True

                    for ele in driver.find_elements(By.XPATH, '//*[@id="contents"]/article/section[2]/div/div[2]/div[2]/ul[2]/li/button'):
                        calender_date = ele.find_element(By.XPATH, './../span').text
                        
                        if int(calender_date) == int(date):
                            logger.info(f'날짜: [{condoNm}-{roomNm}] {int(month)}월{int(date)}일 예약진행합니다.')
                            ele.click()
                            driver, alert = check_alert(logger, driver)
                            
                            if alert is not None:
                                alert.accept()
                                time.sleep(2) # fillout DOM wait time
                                driver.switch_to.window(driver.window_handles[1])
                                driver = fillout(driver)
                                driver, alert = check_alert(logger, driver)
                                
                                if alert.text == '신청하시겠습니까?':
                                    logger.info(f'날짜: [{condoNm}-{roomNm}] {int(month)}월{int(date)}일 예약완료.')
                                    # alert.accept() # 신청하기

                                else:
                                    logger.info(f'{alert.text}')
                                    alert.accept()
                                break
                            
                            else:
                                logger.info('No alert')
                        
                        else:
                            continue
                else:
                    logger.info(f'희망날짜 예약불가능합니다. {desired_dates[index]}')
            
            if flag == False:
                logger.info(f'예약불가능한 날짜가 없습니다. {desired_dates}')

        except Exception as e:
            logger.error(Exception(e))
            raise Exception(e)

        finally:
            return driver
        
    # 로그인 연장 (5분 단위)    
    def login_extend(self, driver):
        try:
            driver.find_element(By.CSS_SELECTOR, '#layerLogin > div.layerbody > div.layerpopcontents > div.btn-grp > button').click()
        
        except NoSuchElementException:
            pass
        
        except Exception as e:
            logger.error(Exception(e))
            raise Exception(e)

        finally:
            return driver
    

In [ ]:
# driver.current_self.url
# driver.refresh()
## page back할때 에러창 뜨는 것 확인
# try:
#     driver.find_elements(By.CLASS_NAME, 'net-error').size()>0
# except Exception:
#     print('a')
#timeServer = 'time.windows.com', any NTP Server domain or ip
# def time_sync(timeServer):
#     c = ntplib.NTPClient() 
#     response = c.request(timeServer, version=3) 
#     print('NTP Server Time과 Local Time과 차이는 %.2f s입니다.' %response.offset)
#     print(ctime(response.tx_time))


In [ ]:
# ## ifttt 연동
# import requests
# ifttt_self.url = 'https://maker.ifttt.com/trigger/{event}/with/key/b3qVzR8-7cXj1ZLqBWf8Nx'.format(event = 'CondoAlert')

# # execute
# set_options();
# login_benepia(self.url, self.id, self.pw);
# choose(year, month, roomNm);

# try:
#     btn_xp = '//*[@id="contents"]/article/section[1]/div/table/tbody/tr[3]/td/a'
#     driver.find_element(By.XPATH, btn_xp).click() 
# #    WebDriverWait(driver, 2).until(EC.alert_is_present())
#     alert = driver.switch_to.alert
#     time.sleep(1)
#     alert.accept()
# except Exception:
#     requests.get(ifttt_self.url, verify = False) # certificate 설정 관련





